# Linear algebra in Julia
Author: Andreas Noack Jensen (MIT) (http://www.econ.ku.dk/phdstudent/noack/)

## Outline
 - Basic linear algebra 
 - Factorizations (Julia Innovation: Matrix Factorization Objects)
 - Special matrix Structures (Matrices know if they are triangular, tridiagonal, etc.)
 - Generic linear algebra (Increasingly Important and totally outside the scope of LAPACK)

### Basic linear algebra
Syntax very similar to other languages but there are some important differences. Define a matrix of random normal variates

In [1]:
A = randn(3,3)

3×3 Array{Float64,2}:
 -0.0106754  0.636594   2.36275 
  0.494222   1.64778   -0.981741
  0.470917   2.33575    1.2516  

In [2]:
B = A
C = copy(A)
[ B C]

3×6 Array{Float64,2}:
 -0.0106754  0.636594   2.36275   -0.0106754  0.636594   2.36275 
  0.494222   1.64778   -0.981741   0.494222   1.64778   -0.981741
  0.470917   2.33575    1.2516     0.470917   2.33575    1.2516  

In [3]:
# Watch out!
A[1]=17
[B C] # B and A are references to the same memory (better for performance)

3×6 Array{Float64,2}:
 17.0       0.636594   2.36275   -0.0106754  0.636594   2.36275 
  0.494222  1.64778   -0.981741   0.494222   1.64778   -0.981741
  0.470917  2.33575    1.2516     0.470917   2.33575    1.2516  

and a vector of ones

In [4]:
x = ones(3)

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

Notice that $A$ has type Array{Float64,2} but $x$ has type Array{Float64,1}. Julia defines the aliases Vector{Type}=Array{Type,1} and Matrix{Type}=Array{Type,2} in Julia. 

Many of the basic operations are the same as in other languages
#### Multiplication

In [5]:
b = A*x

3-element Array{Float64,1}:
 19.9993 
  1.16027
  4.05827

#### Transposition
As in other languages `A'` is the conjugate transpose whereas `A.'` is just the transpose

In [6]:
Asym = A + A'

3×3 Array{Float64,2}:
 34.0      1.13082  2.83367
  1.13082  3.29557  1.35401
  2.83367  1.35401  2.5032 

#### Transposed multiplication
Julia allows us to write this without *

In [7]:
Apd = A'A

3×3 Array{Float64,2}:
 289.466   12.7364   40.271  
  12.7364   8.57616   2.80984
  40.271    2.80984   8.11293

#### Solving linear systems 
The problem $Ax=b$ for square $A$ is solved by the \ function.

In [8]:
A\b

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

#### Least squares
When A is tall the \ function calculates the least squares solution

In [9]:
A[:,1:2]\b

2-element Array{Float64,1}:
 1.13342
 1.12949

The \ function also works for rank deficient least squares problems. In this case, the least squares solution is not unique and Julia returns the solution with smallest norm

In [10]:
A = randn(3,3)

3×3 Array{Float64,2}:
 -0.35972   -1.83175   -0.982013
  1.10692    0.237611   1.77742 
  0.411968   0.241555  -0.531678

In [11]:
[A[:,1] A[:,1]]\b

2-element Array{Float64,1}:
 -1.39005
 -1.39005

#### Underdetermined systems
Minimum norm solution is returned

In [12]:
A[1:2,:]\b[1:2]

3-element Array{Float64,1}:
   2.27193
 -11.8015 
   0.81556

### Factorization
The `\` function hides how the problem is actually solved. Depending on the dimensions of `A`, different methods are chosen to solve the problem. An intermediate step in the solution is to calculate a factorization of the matrix `A`. Basically, a factorization of `A` is a way of expressing `A` as a product of triangular, unitary and permutation matrices. Julia defines a `Factorization` abstract type and several composite subtypes for actually storing factorizations. A `Factorization` object should be thought of as a representation of the matrix `A`.

#### LU

When `A` is square, problem is solved by factorizing the matrix `A=PLU` where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular. Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [13]:
A = randn(3,3)

3×3 Array{Float64,2}:
 -1.06261   -0.0584361  -0.163066
 -0.925962  -0.151315   -0.548002
 -1.16318   -0.157277    0.271689

In [14]:
l,u,p = lu(A)

([1.0 0.0 0.0; 0.913532 1.0 0.0; 0.796058 -0.306342 1.0], [-1.16318 -0.157277 0.271689; 0.0 0.0852416 -0.411263; 0.0 0.0 -0.89027], [3, 1, 2])

In [15]:
norm(l*u - A[p,:])

6.938893903907228e-18

In [16]:
Alu = lufact(A)

Base.LinAlg.LU{Float64,Array{Float64,2}} with factors L and U:
[1.0 0.0 0.0; 0.913532 1.0 0.0; 0.796058 -0.306342 1.0]
[-1.16318 -0.157277 0.271689; 0.0 0.0852416 -0.411263; 0.0 0.0 -0.89027]

The different parts of the factorization can be extracted by special indexing

In [17]:
Alu[:P]

3×3 Array{Float64,2}:
 0.0  0.0  1.0
 1.0  0.0  0.0
 0.0  1.0  0.0

In [18]:
Alu[:L]

3×3 Array{Float64,2}:
 1.0        0.0       0.0
 0.913532   1.0       0.0
 0.796058  -0.306342  1.0

In [19]:
Alu[:U]

3×3 Array{Float64,2}:
 -1.16318  -0.157277    0.271689
  0.0       0.0852416  -0.411263
  0.0       0.0        -0.89027 

We can therefore compute the solution of $Ax=b$ from the factorization

In [20]:
Alu[:U]\(Alu[:L]\(Alu[:P]'b))

3-element Array{Float64,1}:
   3.65174
 -73.1809 
 -22.4586 

However, more importantly the `LU` type allows dispatch and we can solve the system by

In [21]:
Alu\ones(3)

3-element Array{Float64,1}:
 -0.888472
 -0.234398
 -0.258832

This could be useful if the same left-hand-side is used for several right-hand-sides. The factorization can also be used for calculating the determinant because $\det(A)=\det(PLU)=\det(P)\det(U)=\pm \prod u_{ii}$ because $U$ is triangular and the sign is determined from $\det(P)$.

In [22]:
det(A)

0.08827179595455875

In [23]:
Alu = lufact(A)
det(Alu)

0.08827179595455875

#### QR
When `A` is tall, Julia computes the least squares solution $\hat{x}$ that minimizes $\|Ax-b\|_2$. This can be done by factorizing $A=QR$ where $Q$ is unitary/orthogonal and $R=\left(\begin{smallmatrix}R_0\\0\end{smallmatrix}\right)$ and $R_0$ is upper triangular. With the QR factorization the minimum norm can be expressed
\begin{equation*}
\|Ax-b\|=\|QRx-b\|=\|Q(Rx-Q'b)\|=\|Rx-Q'b\|=\left\|\begin{pmatrix}R_0x-Q_0'b\\Q_1'b\end{pmatrix}\right\|=\|R_0x-Q_0'b\|+\|Q_1'b\|
\end{equation*}
and the problem therefore reduces to solving the square problem $R_0x=Q_0'b$ for $x$.

We can QR factorize the submatrix of the two first columns of $A$ by

In [24]:
Aqr = qrfact(A[:,1:2])

Base.LinAlg.QRCompactWY{Float64,Array{Float64,2}} with factors Q and R:
[-0.581473 0.787532 -0.204163; -0.506699 -0.546895 -0.666455; -0.63651 -0.284076 0.717046]
[1.82744 0.210759; 0.0 0.0814117]

`\` has a method for the QR and the least squares problem is therefore solved with

In [25]:
Aqr\b

2-element Array{Float64,1}:
 -27.8788
 171.508 

It should be noted that this is *not* the way `A[:,1:2]\b` is solved. In order to handle rank deficient problems Julia uses a QR factorization with pivoting. Pivoting is enabled with a keyword

In [26]:
Aqrp = qrfact([A[:,1] A[:,1]],Val{true})

Base.LinAlg.QRPivoted{Float64,Array{Float64,2}} with factors Q and R:
[-0.581473 0.795915 0.168549; -0.506699 -0.192205 -0.840424; -0.63651 -0.574088 0.515051]
[1.82744 1.82744; 0.0 2.48253e-16]

Notice that the type is different now. `\` also has a method for `QRPivoted` and the rank deficient problem is therefore computed

In [27]:
Aqrp\b

2-element Array{Float64,1}:
 -4.04941
 -4.04941

Another feature of the QR factorizations is the `Q` types for storing the unitary matrices $Q$. They can be extracted from the different from `QR` types by indexing

In [28]:
Aqr[:Q]

3×3 Base.LinAlg.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.581473   0.787532  -0.204163
 -0.506699  -0.546895  -0.666455
 -0.63651   -0.284076   0.717046

The matrix has a compact internal representation, so indexing is only meaningful if you know how the factorization stores data.

In [29]:
Aqr[:Q][1]

-0.5814728078600326

Even though the matrix is printed as a $3\times 2$ matrix in the factorization object, in practice it can represent the square version as well. Hence both

In [30]:
Aqr[:Q]*ones(2)

3-element Array{Float64,1}:
  0.206059
 -1.05359 
 -0.920586

and

In [31]:
Aqr[:Q]*ones(3)

3-element Array{Float64,1}:
  0.00189586
 -1.72005   
 -0.20354   

works, but not

In [32]:
Aqr[:Q]*ones(4)

LoadError: [91mDimensionMismatch("vector must have length either 3 or 2")[39m

#### Eigendecompositions and the SVD(s)

The results from eigendecompositions and singular value decompositions are also stored in `Factorization` types. This also includes Hessenberg and Schur factorizations.

The eigendecomposition can be computed

In [33]:
AsymEig = eigfact(Asym)

Base.LinAlg.Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}([1.40511, 4.09055, 34.3031], [-0.0523993 0.0839217 -0.995094; -0.560579 -0.827123 -0.040237; 0.826441 -0.55572 -0.0903855])

The values and the vectors can be extracted from the Eigen type by special indexing

In [34]:
AsymEig[:values]

3-element Array{Float64,1}:
  1.40511
  4.09055
 34.3031 

In [35]:
AsymEig[:vectors]

3×3 Array{Float64,2}:
 -0.0523993   0.0839217  -0.995094 
 -0.560579   -0.827123   -0.040237 
  0.826441   -0.55572    -0.0903855

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [36]:
inv(AsymEig)*Asym

3×3 Array{Float64,2}:
  1.0           2.77556e-16  2.08167e-16
  3.21965e-15   1.0          3.33067e-16
 -2.22045e-16  -1.11022e-16  1.0        

Julia also has an `eig` function which returns a tuple with the values and the vectors

In [37]:
eig(Asym)

([1.40511, 4.09055, 34.3031], [-0.0523993 0.0839217 -0.995094; -0.560579 -0.827123 -0.040237; 0.826441 -0.55572 -0.0903855])

We do not recomment this version.

The `svdfact` function computes the singular value decomposition

In [38]:
Asvd = svdfact(A[:,1:2])

Base.LinAlg.SVD{Float64,Float64,Array{Float64,2}}([-0.577464 0.790476; -0.509471 -0.544314; -0.637945 -0.280839], [1.83958, 0.0808745], [0.993389 0.114792; -0.114792 0.993389])

and again `\` has a method for the type enabling least squares by SVD

In [39]:
Asvd\b

2-element Array{Float64,1}:
 -27.8788
 171.508 

Julia does not allow dispatch on the number of output arguments and therefore there are special functions for providing values only: `eigvals` and `svdvals`.

### Special matrix Structures
The structure of matrices is very important in linear algebra. This structure can be made explicit in Julia through composite types. Examples are `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`. Specialized methods are written for the special matrix types to take advantage of their structure. Below some examples are shown

In [40]:
Diagonal(diag(A))

3×3 Diagonal{Float64}:
 -1.06261    ⋅         ⋅      
   ⋅       -0.151315   ⋅      
   ⋅         ⋅        0.271689

In [41]:
LowerTriangular(tril(A))

3×3 LowerTriangular{Float64,Array{Float64,2}}:
 -1.06261     ⋅         ⋅      
 -0.925962  -0.151315   ⋅      
 -1.16318   -0.157277  0.271689

In [42]:
Symmetric(Asym)

3×3 Symmetric{Float64,Array{Float64,2}}:
 34.0      1.13082  2.83367
  1.13082  3.29557  1.35401
  2.83367  1.35401  2.5032 

In [43]:
SymTridiagonal(diag(Asym),diag(Asym,1))

3×3 SymTridiagonal{Float64}:
 34.0      1.13082   ⋅     
  1.13082  3.29557  1.35401
   ⋅       1.35401  2.5032 

When it is known that a matrix is e.g. triangular or symmetric Julia might be able to solve a problem faster by converting the matrix to a special matrix. For some of the procedures, Julia checks if the input matrix is triangular or symmetric and converts the matrix if such a structure is detected. It should noted that `Symmetric`, `Hermitian` and `Triangular` do not copy the input matrix.

#### Symmetric eigenproblem
Whether or not Julia is able to detect if a matrix is symmetric/Hermitian can have a big influence on how fast an eigenvalue problem is solved. Sometimes it is known that a matrix is symmetric or Hermitian but due to floating point errors this is not detected by the `eigvals` function. In following example `Asym1` and `Asym2` are almost identical, but unless Julia is told that `Asym2` is symmetric, the elapsed time for the computation is very different.

In [44]:
n = 1000;
A = randn(n,n);
Asym1 = A + A';
Asym2 = copy(Asym1); Asym2[1,2] += 5eps();
println("Is Asym1 symmetric? ", issymmetric(Asym1))
println("Is Asym2 symmetric? ", issymmetric(Asym2))

Is Asym1 symmetric? true
Is Asym2 symmetric? false


In [45]:
@time eigvals(Asym1);

  0.226750 seconds (18.04 k allocations: 8.860 MiB)


In [46]:
@time eigvals(Asym2);

  0.935519 seconds (24 allocations: 7.921 MiB)


In [47]:
@time eigvals(Symmetric(Asym2));

  0.196412 seconds (1.61 k allocations: 8.074 MiB)


### A big problem
Using tridiagonal matrices makes it possible to work with potentially very large problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a `Matrix` type.

In [48]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.773521 seconds (82.82 k allocations: 187.790 MiB, 11.16% gc time)


7.35792106005476

### Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does. For a long time Julia has also had support for multiplicaton of general element types. Hence, when multiplying integer matrices, the output is also an integer matrix

In [49]:
rand(1:10,3,3)*rand(1:10,3,3)

3×3 Array{Int64,2}:
 132  57  162
 124  67  186
 108  61  173

Recently, more generic linear algebra methods has been added and Julia now supports generic `LU` and `QR` factorizations. Generic eigenvalue and SVD methods have been written more recently (some in external packages).

In general, the `LU` factorization can be computed whenever the matrix element type is closed under the operations `+`, `-`, `*` and `\`. Of course the matrix also has to have full rank. The generic `LU` method in Julia applies pivoting and therefore the element type also has to support `<` and `abs`. Hence it is possible to solve systems of equations of e.g. rational numbers which the following examples show.

#### Example 1: Rational linear system of equations
Julia has rational numbers built in. The following example shows how a linear system of equations can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [50]:
Ar = convert(Matrix{Rational{BigInt}}, rand(1:10,3,3))/10

3×3 Array{Rational{BigInt},2}:
 1//10  7//10  1//1 
 4//5   1//2   7//10
 9//10  3//10  3//5 

In [51]:
x = ones(Int,3)
b = Ar*x

3-element Array{Rational{BigInt},1}:
 9//5
 2//1
 9//5

In [52]:
Ar\b

3-element Array{Rational{BigInt},1}:
 1//1
 1//1
 1//1

In [53]:
lufact(Ar)

Base.LinAlg.LU{Rational{BigInt},Array{Rational{BigInt},2}} with factors L and U:
Rational{BigInt}[1//1 0//1 0//1; 8//1 1//1 0//1; 9//1 20//17 1//1]
Rational{BigInt}[1//10 7//10 1//1; 0//1 -51//10 -73//10; 0//1 0//1 16//85]

#### Example 2: Rational matrix from eigenstructure
The next example shows how rational matrix arithmetic can be used for calculating a matrix given rational eigenvalues and -vectors. I have found this convenient when giving examples of linear dynamic systems.

In [54]:
λ1,λ2,λ3 = 1//1,1//2,1//4
v1,v2,v3 = [1,0,0],[1,1,0],[1,1,1]
V,Λ = [v1 v2 v3], Diagonal([λ1,λ2,λ3])
A = V*Λ/V

3×3 Array{Rational{Int64},2}:
 1//1  -1//2  -1//4
 0//1   1//2  -1//4
 0//1   0//1   1//4